In [5]:
import numpy as np
from sympy.physics.quantum import TensorProduct
from qiskit.circuit.library import RealAmplitudes
from qiskit import ClassicalRegister, QuantumRegister, QuantumCircuit
from qiskit import Aer, transpile, assemble
from qiskit.circuit import ParameterVector
from qiskit.providers.ibmq.managed import IBMQJobManager

General Ansatz Generator

In [3]:
#multi-controlled u3
#Parameters: param (list); num_control_qubits (int)
#Return: The multi-controlled u3 Gate object
def mtcu(param, num_control_qubits):
    
    Qbit = QuantumRegister(1)
    qc = QuantumCircuit(Qbit)
    qc.u3(param[0], param[1], param[2], Qbit[0])
    
    #gate_label = 'U3({para1}, {para2}, {para3})'.format(para1 = param[0], para2 = param[1], 
                                                                       #para3 = param[2])
    gate_label = 'U3'
    u3 = qc.to_gate(label = gate_label)
    
    
    #control part
    cu3 = u3.control(num_control_qubits)
    
    return cu3

#Append multi-controlled u3 advance to the circuit depends on the bit-string label of the gate
#Parameters: 
    #param(2D list, size (num_qubits * 3))
    #num_ctrlqubits (int)
    #circuit (Circuit Object)
    #ctrlqubits (list)
    #targetqubit (list)
    #bit_str (string)
def mtcu_advance(param, num_ctrlqubits, circuit, ctrlqubits, targetqubits, bit_str):
    cu3 = mtcu(param, num_ctrlqubits)
    
    for i in range(len(bit_str)):
        if bit_str[i] == '0':
            circuit.x(ctrlqubits[i])
    
    circuit.append(cu3, ctrlqubits + targetqubits)
    
    for i in range(len(bit_str)):
        if bit_str[i] == '0':
            circuit.x(ctrlqubits[i])

In [28]:
#multi-controlled RY
#Parameters: param (list); num_control_qubits (int)
#Return: The multi-controlled RY Gate object
def mtcry(theta, num_control_qubits):
    
    Qbit = QuantumRegister(1)
    qc = QuantumCircuit(Qbit)
    qc.ry(theta, Qbit[0])
    
    #gate_label = 'U3({para1}, {para2}, {para3})'.format(para1 = param[0], para2 = param[1], 
                                                                       #para3 = param[2])
    gate_label = 'RY({})'.format(theta)
    RY = qc.to_gate(label = gate_label)
    
    
    #control part
    mtcry = RY.control(num_control_qubits)
    
    return mtcry


#Append multi-controlled u3 advance to the circuit depends on the bit-string label of the gate
#Parameters: 
    #param(2D list, size (num_qubits * 3))
    #num_ctrlqubits (int)
    #circuit (Circuit Object)
    #ctrlqubits (list)
    #targetqubit (list)
    #bit_str (string)
def mtcry_advance(theta, num_ctrlqubits, circuit, ctrlqubits, targetqubits, bit_str):
    mtcry_gate = mtcry(theta, num_ctrlqubits)
    
    for i in range(len(bit_str)):
        if bit_str[i] == '0':
            circuit.x(ctrlqubits[i])
    
    circuit.append(mtcry_gate, ctrlqubits + targetqubits)
    
    for i in range(len(bit_str)):
        if bit_str[i] == '0':
            circuit.x(ctrlqubits[i])


In [34]:
#generate general non-parametrized ansatz 
#parameters:
    #param (2D list, size num_qubits * 3) : parameters to generate general ansatz
    #qbits (QuantumRegister object)
    #cbits (ClassicalRegister object)
#return: the ansatz (Circuit object)
def general_ansatz(param, qbits, cbits):
    num_qubits = len(qbits)
    qc = QuantumCircuit(qbits, cbits)
    
    #append U3_0 (without control) on the first qubit 
    qc.u3(param[0][0], param[0][1], param[0][2], qbits[len(qbits) - 1 ])
    
    #setup Controlled-U3 
    for i in range(1, num_qubits):
        num_ctrlqubits = i
        #list of ctrlqubits' indexes
        ctrlqubits = [i for i in range(num_qubits - num_ctrlqubits, num_qubits)]
        #list of targetqubits' indexes
        targetqubits = [num_qubits -1 -num_ctrlqubits]
        #generate bit_string list from the number of controlqubits
        bit_str_lst = bit_str_list(num_ctrlqubits)
        #generate sorted bit_string list from the number of controlqubits
        bit_str_lst_sorted = sorted(bit_str_lst)
        
        for j in range(len(bit_str_lst)):
            
            bit_str = bit_str_lst[j]
            gate_idx = bit_str_to_dec('1'+ bit_str_lst_sorted[j]) - 1
            mtcu_advance(param[gate_idx], num_ctrlqubits, qc, ctrlqubits, targetqubits, bit_str)
            
    return qc

#generate general non-parametrized ansatz for Ising model
#parameters:
    #param (2D list, size num_qubits * 3) : parameters to generate general ansatz
    #qbits (QuantumRegister object)
    #cbits (ClassicalRegister object)
#return: the ansatz (Circuit object)

def ising_general_ansatz(theta_list, qbits, cbits):
    num_qubits = len(qbits)
    qc = QuantumCircuit(qbits, cbits)
    
    #append U3_0 (without control) on the first qubit 
    qc.ry(theta_list[0], num_qubits - 1)
    
    #setup Controlled-U3 
    for i in range(1, num_qubits):
        num_ctrlqubits = i
        #list of ctrlqubits' indexes
        ctrlqubits = [i for i in range(num_qubits - num_ctrlqubits, num_qubits)]
        #list of targetqubits' indexes
        targetqubits = [num_qubits -1 -num_ctrlqubits]
        #generate bit_string list from the number of controlqubits
        bit_str_lst = bit_str_list(num_ctrlqubits)
        #generate sorted bit_string list from the number of controlqubits
        bit_str_lst_sorted = sorted(bit_str_lst)
        
        for j in range(len(bit_str_lst)):
            
            bit_str = bit_str_lst[j]
            gate_idx = bit_str_to_dec('1'+ bit_str_lst_sorted[j]) - 1
            mtcry_advance(theta_list[gate_idx], num_ctrlqubits, qc, ctrlqubits, targetqubits, bit_str)
            
    return qc


#parameterized general ansatz
def ising_parameterized_general_ansatz(num_qubits):
    
    theta_list = ParameterVector('theta', 2 ** num_qubits - 1)
    
    qc = QuantumCircuit(num_qubits, num_qubits)
    
    qc.u3(theta_list[0], 0, 0, [num_qubits - 1])
    
    for i in range(1, num_qubits):
        num_ctrlqubits = i
        #list of ctrlqubits' indexes
        ctrlqubits = [i for i in range(num_qubits - num_ctrlqubits, num_qubits)]
        #list of targetqubits' indexes
        targetqubits = [num_qubits -1 -num_ctrlqubits]
        #generate bit_string list from the number of controlqubits
        bit_str_lst = bit_str_list(num_ctrlqubits)
        #generate sorted bit_string list from the number of controlqubits
        bit_str_lst_sorted = sorted(bit_str_lst)
        
        for j in range(len(bit_str_lst)):
            
            bit_str = bit_str_lst[j]
            gate_idx = bit_str_to_dec('1'+ bit_str_lst_sorted[j]) - 1
            mtcu_advance([theta_list[gate_idx],0,0], num_ctrlqubits, qc, ctrlqubits, targetqubits, bit_str)
            
    return qc
    

In [30]:
#generate list of bit_string that can be formed by n qubits (the ordered is not sorted by least to greatest)
#parameters: n (int)
#return: the list of bit strings
def bit_str_list(n):
    bit_str_lst = []
    
    for i in range( 2**n ):
        bit_str = ''
        rev_bit_str = ''
        temp = i
        
        for j in range(n):
            rev_bit_str += str(temp % 2)
            temp = temp // 2
        
        bit_str_lst.append(rev_bit_str)
    return bit_str_lst

#convert bit_str form to decimal form
#parameters: bit_str (string)
#return: the decimal form of bit_str (int)
def bit_str_to_dec(bit_str):
    dec = 0
    for i in range(len(bit_str)-1, -1,-1):
        bit = int(bit_str[i])
        dec += bit * ( 2 ** (len(bit_str) - 1 - i) )
    return dec

In [31]:
def final_coeff(bit_str, theta_list):
    coeff = 1
    for i in range(len(bit_str)-1, -1, -1):
        code = '1' + bit_str[:i]
        num = bit_str_to_dec(code) - 1
        
        if bit_str[i] == '0':
            coeff *= np.cos(theta_list[num]/2)
        else:
            coeff *= np.sin(theta_list[num]/2)
    return coeff

def theor_zz_probs(theta_list):
    count = {}
    n = (np.log2(len(theta_list) + 1))
    if int(n) == n:
        num_qubits = int(n)
        for element in bit_str_list(num_qubits):
            count[element] = final_coeff(element, theta_list) ** 2
        return count
    else:
        print('invalid theta_list')
        
def theor_x_probs(theta_list):
    H = 1/ np.sqrt(2) * np.array([[1,1], [1, -1]])
    
    def tensorproduct_via_list(lst):
        product = lst[0]
        for i in range(len(lst)-1):
            product = TensorProduct(product, lst[i+1])
        return product
    
    count = {}
    n = (np.log2(len(theta_list) + 1))
    
    if int(n) == n:
        
        num_qubits = int(n)
        bit_string_list = bit_str_list(num_qubits)
        bit_string_list.sort()
        
        x_prob_vec = []
        for element in bit_string_list:
            x_prob_vec.append([final_coeff(element, theta_list)])
        
        prob_vec = np.matmul(tensorproduct_via_list([H] * num_qubits), x_prob_vec)
        
        for i in range(len(prob_vec)):
            count[bit_string_list[i]] = (prob_vec[i][0])**2
    
    else:
        print('invalid theta_list')
    
    return count

Measurement 

In [32]:
#generate z-measurement  circuit and x-measurement circuit knowing type of ansatz, parameters and qubits number
#parameters: 
    #ansatz_name (str) 
        #'General' : General Ansatz
        #'RealAmp_full' : Real Amplitudes Ansatz with full entanglement
        #'RealAmp_linear' : Real Amplitudes Ansatz with linear entanglement
        #'RealAmp_circular' : Real Amplitudes Ansatz with circular entanglement
    #theta_list (list): list of circuit's parameters
    #num_qubits (int) : number of qubits 
#return: tuple (z_qc, x_qc)

def measurecircuit(theta_list, num_qubits, ansatz_name):
    #generate qubits for measure circuit
    qbits = QuantumRegister(num_qubits)
    cbits = ClassicalRegister(num_qubits)
    
    
    if ansatz_name == 'General':
        
        #generate param matrix from theta_list
        #param = [3 * [0] for _ in range(( 2 ** num_qubits - 1))]
        #for i in range(len(param)):
        #    param[i][0] = theta_list[i]
        
        
        x_qc = ising_general_ansatz(theta_list, qbits, cbits)
        x_qc.h(qbits[:])
        x_qc.measure(qbits[:], cbits[:])
        
        
        
        z_qc = ising_general_ansatz(theta_list, qbits, cbits)
        z_qc.measure(qbits[:], cbits[:]) 
    
    elif ansatz_name == 'RealAmp_full':
        
        ansatz = RealAmplitudes(num_qubits, reps = 1, entanglement = 'full')
        ansatz.assign_parameters(theta_list, inplace = True)
        
        #x-measure circuit
        x_qc = QuantumCircuit(qbits, cbits)
        x_qc.append(ansatz, [i for i in range(num_qubits)])
        x_qc.h(qbits[:])
        x_qc.measure(qbits[:], cbits[:])
        
        #z-measure circuit
        z_qc = QuantumCircuit(qbits, cbits)
        z_qc.append(ansatz, [i for i in range(num_qubits)])
        z_qc.measure(qbits[:], cbits[:])
        
    elif ansatz_name == 'RealAmp_linear':
        
        ansatz = RealAmplitudes(num_qubits, reps = 1, entanglement = 'linear')
        ansatz.assign_parameters(theta_list, inplace = True)
        
        #x-measure circuit
        x_qc = QuantumCircuit(qbits, cbits)
        x_qc.append(ansatz, [i for i in range(num_qubits)])
        x_qc.h(qbits[:])
        x_qc.measure(qbits[:], cbits[:])
        
        #z-measure circuit
        z_qc = QuantumCircuit(qbits, cbits)
        z_qc.append(ansatz, [i for i in range(num_qubits)])
        z_qc.measure(qbits[:], cbits[:])
        
    elif ansatz_name == 'RealAmp_circular':
        
        ansatz = RealAmplitudes(num_qubits, reps = 1, entanglement = 'circular')
        ansatz.assign_parameters(theta_list, inplace = True)
    
        #x-measure circuit
        x_qc = QuantumCircuit(qbits, cbits)
        x_qc.append(ansatz, [i for i in range(num_qubits)])
        x_qc.h(qbits[:])
        x_qc.measure(qbits[:], cbits[:])
        
        #z-measure circuit
        z_qc = QuantumCircuit(qbits, cbits)
        z_qc.append(ansatz, [i for i in range(num_qubits)])
        z_qc.measure(qbits[:], cbits[:]) 
        
    else: 
        print('Invalid input for parameter \'ansatz_name\'')
    
    
    return [z_qc, x_qc]

In [64]:
#compute the expectation value of the Hamiltonian of the Ising system from the result
#parameter:
    #num_qubits (int): number of qubits
    #h (float): value of the magnetic coefficient
    #zz_counts (dictionary): results after zz-measurement
    #x_counts (dictionary) : results after x-measuremnt
    #shots (int): number of shots
def hamiltonian_measure(num_qubits, h, zz_counts, x_counts, shots=10000): 
    
    zz_probs = {}
    for key in zz_counts:
        zz_probs[key] = zz_counts[key]/ shots
    
    def zz_measure(num_qubits, zz_probs):
        
        zz_exp = 0
        for i in range(num_qubits):
            zz_exp_i = 0
            if i == num_qubits -1:
                for key in zz_probs:
                    if key[0] == key[-1]:
                        sign = +1
                    else:
                        sign = -1
                    zz_exp_i += sign * (zz_probs[key])
            
            else:
                for key in zz_probs:
                    if key[i] == key[i+1]:
                        sign = +1
                    else:
                        sign = -1
                    zz_exp_i += sign * (zz_probs[key])
            zz_exp += zz_exp_i
        
        return - zz_exp

    x_probs = {}
    for key in x_counts:
        x_probs[key] = x_counts[key]/ shots
    
    def x_measure(num_qubits, h, x_probs):
        
        x_exp = 0
        for i in range(num_qubits):
            x_exp_i = 0
            for key in x_probs:
        
                if key[i] == '0':
                    sign = +1
                else:
                    sign = -1
            
                x_exp_i += sign * x_probs[key]
            
            x_exp += x_exp_i
    
        return - h * x_exp
    return(zz_measure(num_qubits, zz_probs) + x_measure(num_qubits, h, x_probs))


In [40]:
#objective_function/ Cost function of this Problem, which is the expectation value of Ising Model's hamiltonian
#parameters:
    #theta_list (list of real theta parameters- 1st parameter of each multicontrolled U3)
    #num_qubits (int)
    #h (float)
    #backend : Backend object
#return: the value of the function (float)
def ising_costfun(theta_list, num_qubits, h, ansatz_name, backend = Aer.get_backend('qasm_simulator'), real_qcomp = False ):
    
    #get backend
    backend = backend
    numshots = 10000
    
    if real_qcomp:
        circs = measurecircuit(theta_list, num_qubits, ansatz_name)
        circs_transpile = transpile(circs, backend = backend)
    
        job_manager = IBMQJobManager()
        results = job_manager.run(circs_transpile, backend = backend, shots = numshots).results()
    
        zz_counts = results.get_counts(0)
        x_counts = results.get_counts(1)
    
    else: 
        #measure term ZZ
        zz_qc = measurecircuit(theta_list, num_qubits,ansatz_name)[0]
        
        #if ansatz_name != 'General': 
        #    zz_qc_transpile = transpile(zz_qc, backend = backend)
        #else:
        #    zz_qc_transpile = zz_qc
        
        zz_qc_transpile = transpile(zz_qc, backend = backend)
        zz_job = assemble(zz_qc_transpile, backend = backend, shots = numshots)
        zz_result = backend.run(zz_job).result()
        zz_counts = zz_result.get_counts()
    
        #measure term X
    
        x_qc = measurecircuit(theta_list, num_qubits, ansatz_name)[1]
        #if ansatz_name != 'General': 
        #    x_qc_transpile = transpile(x_qc, backend = backend)
        #else:
        #    x_qc_transpile = x_qc
        
        x_qc_transpile = transpile(x_qc, backend = backend)
        x_job = assemble(x_qc_transpile, backend = backend, shots = numshots)
        x_result = backend.run(x_job).result()
        x_counts = x_result.get_counts()
    
    #measure expectation value of hamiltonian
    h_exp = hamiltonian_measure(num_qubits, h, zz_counts, x_counts)
    
    return h_exp

In [167]:
def theor_ising_costfun(theta_list, num_qubits, h):
    zz_probs = theor_zz_probs(theta_list)
    x_probs = theor_x_probs(theta_list)
    
    def zz_measure(num_qubits, zz_probs):
        
        zz_exp = 0
        for i in range(num_qubits):
            zz_exp_i = 0
            if i == num_qubits -1:
                for key in zz_probs:
                    if key[0] == key[-1]:
                        sign = +1
                    else:
                        sign = -1
                    zz_exp_i += sign * (zz_probs[key])
            
            else:
                for key in zz_probs:
                    if key[i] == key[i+1]:
                        sign = +1
                    else:
                        sign = -1
                    zz_exp_i += sign * (zz_probs[key])
            zz_exp += zz_exp_i
        
        return - zz_exp
    
    def x_measure(num_qubits, h, x_probs):
        
        x_exp = 0
        for i in range(num_qubits):
            x_exp_i = 0
            for key in x_probs:
        
                if key[i] == '0':
                    sign = +1
                else:
                    sign = -1
            
                x_exp_i += sign * x_probs[key]
            
            x_exp += x_exp_i
    
        return - h * x_exp
    
    h_exp = (zz_measure(num_qubits, zz_probs) + x_measure(num_qubits, h, x_probs))
    
    return h_exp

Getting data from files

In [168]:
#get_E_range from E_range file and number of qubits
def get_E_range(filename, n):
    with open(filename, 'r') as infile:
        for i in range(1, n+1):
            infile.readline()
            line = infile.readline().split(' ')
    E_range = []
    for element in line:
        E_range.append(complex(element))
        
    return E_range